#Team Chat

In [ ]:
print("hellooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooo")

hellooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooo


In [ ]:
print("Howu is!!!!!!!!!!")

Howu is!!!!!!!!!!


In [ ]:
print("Woooooooo ( ﾉ･o･ )ﾉ")

Woooooooo ( ﾉ･o･ )ﾉ


In [ ]:
print("sdaf")

sdaf


#Libraries

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import re
from google.colab import files
from sklearn import svm
#from sklearn.impute import SimpleImputer
#KNNImputer, IterativeImputer
#from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error

import warnings
warnings.filterwarnings("ignore")

#Loading Data

In [ ]:
test_url = "https://raw.githubusercontent.com/NoorAbuGhz/WiDS-2024/4f7161a684f1bf10d4f1c5a50ef811372dd5fa96/test.csv"
train_url = "https://raw.githubusercontent.com/NoorAbuGhz/WiDS-2024/4f7161a684f1bf10d4f1c5a50ef811372dd5fa96/train.csv"
solution_template_url = "https://raw.githubusercontent.com/NoorAbuGhz/WiDS-2024/4f7161a684f1bf10d4f1c5a50ef811372dd5fa96/solution_template.csv"

test = pd.read_csv(test_url)
train = pd.read_csv(train_url)
sample = pd.read_csv(solution_template_url)

patients = test['patient_id']

#Data Exlporation

In [ ]:
train['patient_id'].nunique()

13173

In [ ]:
print(train.shape)
print(test.shape)

(13173, 152)
(5646, 151)


In [ ]:
train['metastatic_first_novel_treatment'].unique()

array([nan, 'OLAPARIB', 'PEMBROLIZUMAB'], dtype=object)

In [ ]:
# Checking number of unique values per column
# for col in test.columns:
#   print(col,"\t",train[col].nunique())

In [ ]:
print(train.patient_race.unique())

# patient_race - Asian, African American, Hispanic or Latino, White, Other Race
# Columns related to race: race_white, race_black, race_asian, race_native, race_pacific, race_other, race_multiple, hispanic

# Asian - race_asian
# African American - race_black
# Hispanic or Latino - hispanic
# Asian - race_asian
# Other - race_other, race_multiple, race_native, race_pacific

[nan 'White' 'Hispanic' 'Black' 'Other' 'Asian']


In [ ]:
# Checking columns with null
# for col in test.columns:
#   if train[col].isnull().sum() > 0:
#     print(col,"\t",train[col].isnull().sum())

# 13173 total rows

In [ ]:
s = test['income_household_median']
s.describe()

count      5646.000000
mean      73712.310602
std       20225.771198
min       32935.380000
25%       61075.130000
50%       69407.350000
75%       82970.380000
max      164119.200000
Name: income_household_median, dtype: float64

#Dropping Columns

In [ ]:
# train=train.drop(train.loc[:, 'age_under_10':'age_over_80'].columns, axis=1)
# train=train.drop(train.loc[:, 'married':'income_household_15_to_20'].columns, axis=1)
# train=train.drop(train.loc[:, 'income_household_20_to_25':'income_individual_median'].columns, axis=1)
# train=train.drop(train.loc[:, 'home_ownership':'labor_force_participation'].columns, axis=1)


train=train.drop(train.loc[:, 'unemployment_rate':'farmer'].columns, axis=1)
# train=train.drop(train.loc[:, 'limited_english':'veteran'].columns, axis=1)
# after dropping the score got a little better (0.09)
train=train.drop(train.loc[:, 'Average of Jan-13':'Average of Dec-18'].columns, axis=1)

# id
# train=train.drop(['patient_id'], axis=1)
# a lot of null
train=train.drop(['payer_type','bmi'], axis=1)
# basically empty
train=train.drop(['metastatic_first_novel_treatment','metastatic_first_novel_treatment_type'], axis=1)
# one unique
train=train.drop(['patient_gender'], axis=1)

In [ ]:
# test=test.drop(test.loc[:, 'age_under_10':'age_over_80'].columns, axis=1)
# test=test.drop(test.loc[:, 'married':'income_household_15_to_20'].columns, axis=1)
# test=test.drop(test.loc[:, 'income_household_20_to_25':'income_individual_median'].columns, axis=1)
# test=test.drop(test.loc[:, 'home_ownership':'labor_force_participation'].columns, axis=1)


test=test.drop(test.loc[:, 'unemployment_rate':'farmer'].columns, axis=1)
# test=test.drop(test.loc[:, 'limited_english':'veteran'].columns, axis=1)
test=test.drop(test.loc[:, 'Average of Jan-13':'Average of Dec-18'].columns, axis=1)

# id
# test=test.drop(['patient_id'], axis=1)
# a lot of null
test=test.drop(['payer_type','bmi'], axis=1)
# basically empty
test=test.drop(['metastatic_first_novel_treatment','metastatic_first_novel_treatment_type'], axis=1)
# one unique
test=test.drop(['patient_gender'], axis=1)


#Filling NA (Categorical/Numerical)

In [ ]:
# Filling null values in patient race based on the races of residents in their area
# Made the score worse -Noor
# I made a change in the second for-loop and now it's making the score better -Khaled

race_fill_value = {
    'race_asian': 'Asian',
    'race_black': 'Black',
    'hispanic': 'Hispanic',
    'race_white': 'White',
}

for row in train[train.patient_race.isnull()].index:
    majority_race = pd.to_numeric(train.loc[row, 'race_white':'hispanic'], errors='coerce').idxmax()
    train.loc[row, 'patient_race'] = race_fill_value.get(majority_race, 'Other')


# Repeating it on test
# Here it was test[train.patient_race.isnull()].index -Khaled
for row in test[test.patient_race.isnull()].index:
    majority_race = pd.to_numeric(test.loc[row, 'race_white':'hispanic'], errors='coerce').idxmax()
    test.loc[row, 'patient_race'] = race_fill_value.get(majority_race, 'Other')

In [ ]:
cats = train.select_dtypes(include=['object']).columns.tolist()
nums = [x for x in train.columns if x not in cats][:-1]

In [ ]:
train[nums] = train[nums].fillna(train[nums].mean())
test[nums] = test[nums].fillna(test[nums].mean())

train[cats] = train[cats].fillna(train[cats].mode().iloc[0])
test[cats] = test[cats].fillna(test[cats].mode().iloc[0])

#Fixing Inconsistency

In [ ]:
train['breast_cancer_diagnosis_desc'].replace({'Malig neoplasm of upper-outer quadrant of right male breast':\
                                               'Malignant neoplasm of upper-outer quadrant of right female breast', \
                                               'Malignant neoplasm of unspecified site of left male breast':\
                                               'Malignant neoplasm of unspecified site of left female breast', \
                                               'Malig neoplasm of upper-inner quadrant of right male breast':\
                                               'Malignant neoplasm of upper-inner quadrant of right female breast', \
                                               'Malignant neoplasm of central portion of left male breast':\
                                               'Malignant neoplasm of central portion of left female breast'}, inplace=True)

In [ ]:
# thanks @iqbalsyahakbar
# thanks THIAGO MANTUANI
rep = {
    'malig ' : '',
    'malignant ' : '',
    'neoplm ' : '',
    'neoplasm ' : '',
    'unsp ' : 'unspecified ',
    'ovrlp' : 'overlapping',
    'sites' : 'site',
    'site,' : 'site',
    'breast,' : 'breast',
    'areola,' : 'areola',
    '(female),' : 'female',
    'of ' : '',
    ' and ' : ' ',
    '-' : ' '
}
rep = dict((re.escape(k), v) for k, v in rep.items())
pattern = re.compile("|".join(rep.keys()))

train['breast_cancer_diagnosis_desc'] = train.breast_cancer_diagnosis_desc.astype('str').apply(
    lambda x: pattern.sub(lambda m: rep[re.escape(m.group(0))], x.lower())
)
test['breast_cancer_diagnosis_desc'] = test.breast_cancer_diagnosis_desc.astype('str').apply(
    lambda x: pattern.sub(lambda m: rep[re.escape(m.group(0))], x.lower())
)

#train['breast_cancer_diagnosis_desc'].unique()

In [ ]:
# I tried sth but it didn't work :,)  -Noor
# I changed some stuff and now it's working -Khaled
# It keeps saying 'patient_state' and 'breast_cancer_diagnosis_code' doesnt exist
# And that the number of rows in train and test don't match


# # thanks simonagradinaru

# Changing some states to match the correct zipcodes
train['patient_state'] = np.where(train['patient_zip3'] == 630, 'MO', np.where(train['patient_zip3'] == 864, 'AZ', train['patient_state']))

# Fixing the Division Missouri belongs to
train['Division'] = np.where(train['patient_state'] == 'MO', 'West North Central', train['patient_state'])

# Fixing inconsistencies in population for some patients
pop_cols = train.loc[:, 'population':'veteran'].columns.to_list()

train.loc[train.patient_id == 714510, pop_cols] = train.loc[train.patient_id == 636245, pop_cols].values
train.loc[train.patient_id == 271422, pop_cols] = train.loc[train.patient_id == 271245, pop_cols].values
train.loc[train.patient_id == 441322, pop_cols] = train.loc[train.patient_id == 982003, pop_cols].values

#  Changing the 3 digit breast_cancer_diagnosis_code to be 4 digits
train['breast_cancer_diagnosis_code'] = train['breast_cancer_diagnosis_code'].replace({'C509': 'C5091'})


#### Repeating the steps for test

test['patient_state'] = np.where(test['patient_zip3'] == 630, 'MO', np.where(test['patient_zip3'] == 864, 'AZ', test['patient_state']))

train['Division'] = np.where(train['patient_state'] == 'MO', 'West North Central', train['patient_state'])

pop_cols = test.loc[:, 'population':'veteran'].columns.to_list()

test.loc[test.patient_id == 714510, pop_cols] = test.loc[test.patient_id == 636245, pop_cols].values
test.loc[test.patient_id == 271422, pop_cols] = test.loc[test.patient_id == 271245, pop_cols].values
test.loc[test.patient_id == 441322, pop_cols] = test.loc[test.patient_id == 982003, pop_cols].values

test['breast_cancer_diagnosis_code'] = test['breast_cancer_diagnosis_code'].replace({'C509': 'C5091'})

In [ ]:
# Dropping id
train=train.drop(['patient_id'], axis=1)
test=test.drop(['patient_id'], axis=1)

#Converting Categorical to Numerical

In [ ]:
print(train.shape)
print(test.shape)

(13173, 71)
(5646, 70)


In [ ]:
encoded_data_train = pd.get_dummies(train[cats])
                          #drop_first = True)
train = pd.concat([train, encoded_data_train], axis=1)

encoded_data_test = pd.get_dummies(test[cats])
test = pd.concat([test, encoded_data_test], axis=1)

In [ ]:
# # This helped with the score - Khaled

from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()

for i in cats:
  train[i + '_encoded'] = label_encoder.fit_transform(train[i])
  test[i + '_encoded'] = label_encoder.fit_transform(test[i])

train = train.drop(cats, axis=1)
test = test.drop(cats, axis=1)

In [ ]:
cats

['patient_race',
 'patient_state',
 'Region',
 'Division',
 'breast_cancer_diagnosis_code',
 'breast_cancer_diagnosis_desc',
 'metastatic_cancer_diagnosis_code']

In [ ]:
print(train.shape)
print(test.shape)

(13173, 300)
(5646, 251)


#Creating Missing Columns (one-hot) in Train/Test

In [ ]:
df = pd.concat([train.drop(columns="metastatic_diagnosis_period"), test])

cols_not_common = []
for index, value in df.isna().sum().items():
    if value > 0:
        #print(index, value)
        cols_not_common.append(index)

for col in cols_not_common:
  if col not in train.columns:
    train[col] = 0
  if col not in test.columns:
    test[col] = 0

test = test.reindex(columns=sorted(test.columns))
train = train.reindex(columns=sorted(train.columns))

df.fillna(0, inplace=True)

#Removing Duplicate rows

In [ ]:
# I changed the model used from random forest to catboost and it gave the exact same score
# And both models had missing values in the submission file - Khaled

# train.drop_duplicates(inplace=True)
# test.drop_duplicates(inplace=True)

# Dropping rows seems to always make the score worse - Khaled

In [ ]:
print(train.duplicated().sum(), train.shape)
print(test.duplicated().sum(), test.shape)

22 (13173, 311)
36 (5646, 310)


In [ ]:
cor_train = train.corr()

target = 'metastatic_diagnosis_period'
alpha = 0.1
cor_target = abs(cor_train[target])

relevant_features = cor_target[cor_target>alpha]
print(len(relevant_features))
relevant_features
# I think that these correlations are really bad, especially as the highest one is 0.536860

17


breast_cancer_diagnosis_code_1744                                          0.207609
breast_cancer_diagnosis_code_1748                                          0.191176
breast_cancer_diagnosis_code_1749                                          0.498996
breast_cancer_diagnosis_code_C50411                                        0.105243
breast_cancer_diagnosis_code_C50911                                        0.136724
breast_cancer_diagnosis_code_C50912                                        0.145692
breast_cancer_diagnosis_code_C50919                                        0.132038
breast_cancer_diagnosis_code_encoded                                       0.536860
breast_cancer_diagnosis_desc_breast female unspecified                     0.498996
breast_cancer_diagnosis_desc_encoded                                       0.407334
breast_cancer_diagnosis_desc_other specified site female breast            0.191176
breast_cancer_diagnosis_desc_unspecified site left female breast           0

In [ ]:
# train[relevant_features.index]

#Model Creation

In [ ]:
x = train.drop('metastatic_diagnosis_period', axis = 1)
y = train['metastatic_diagnosis_period']

In [ ]:
from sklearn.ensemble import RandomForestRegressor

random_forest = RandomForestRegressor(n_estimators=100,
                                max_depth=10,
                                min_samples_split=2,
                                min_samples_leaf=1,
                                max_features='auto',
                                bootstrap=True,
                                random_state=42,
                                criterion='friedman_mse',
                                min_impurity_decrease=0.0,
                                oob_score=False)

# Train the model
random_forest.fit(x, y)

# Make predictions
predictions = random_forest.predict(x)

# # Calculate accuracy
# accuracy = accuracy_score(y, predictions)
# print("Accuracy:", accuracy)

mse = mean_squared_error(y, predictions, squared=False)
print("Mean Squared Error:", mse)

Mean Squared Error: 72.75526537803995


In [ ]:
print(predictions)

[ 81.43103479  58.05409521 222.81583615 ...  60.69658018 274.40427537
  50.35090041]


In [ ]:
import xgboost as xgb

# Instantiate the XGBoost regressor
params = {
    'objective': 'reg:squarederror',  # Objective function for regression
    'colsample_bytree': 0.3,          # Fraction of features to sample for each tree
    'learning_rate': 0.01,            # Step size shrinkage used in update to prevent overfitting
    'max_depth': 20,                  # Maximum depth of a tree
    'alpha': 10,                      # L1 regularization term on weights
    'n_estimators': 50,               # Number of boosting rounds (trees)
    'min_child_weight': 2,            # Minimum sum of instance weight needed in a child
    'gamma': 0.1,                     # Minimum loss reduction required to make a further partition on a leaf node
    'subsample': 0.95                  # Fraction of samples used in each boosting round
    #'reg_lambda': 1.0                 # L2 regularization term on weights
}

# Instantiate the XGBoost regressor
xgb_reg = xgb.XGBRegressor(**params)
# Fit the model to the training data
xgb_reg.fit(x, y)

# Make predictions on the testing data
xboost_pred = xgb_reg.predict(x)

# Evaluate the model
mse2 = mean_squared_error(y, xboost_pred, squared=False)
print("Mean Squared Error:", mse2)

Mean Squared Error: 85.76629180408989


In [ ]:
!pip install catboost

In [ ]:
from catboost import CatBoostRegressor

params = {
    'iterations': 100,                # Number of trees to build (equivalent to n_estimators)
    'depth': 7,                       # Depth of the trees,6                                                           - I got stupid and put the depth at 15. Wouldn't recommend.
    'learning_rate': 0.1,             # Step size shrinkage used in update to prevent overfitting
    'l2_leaf_reg': 4,                 # L2 regularization term on leaf weights
    'bagging_temperature': 1.0,       # Controls the intensity of the temperature (the higher the value, the more aggressive the bagging)
    'random_strength': 1.0,           # The strength of the random permutation that is used to make the splits
    'border_count': 254,              # Approximation parameter for dealing with categorical features
    'thread_count': 4,                # The number of parallel threads used to train the model,
    'verbose': False                  # Whether to print information during training
}

# Instantiate the CatBoost regressor
catboost_reg = CatBoostRegressor(**params)

# Fit the model to the training data
catboost_reg.fit(x, y)

# Make predictions on the testing data
cat_pred = catboost_reg.predict(x)

# Evaluate the model
mse = mean_squared_error(y, cat_pred, squared=False)
print("Root Mean Squared Error:", mse)

Root Mean Squared Error: 78.61246857872068


#Predicting

In [ ]:
# Catboost seems to give the best results so far -Noor
y_test = catboost_reg.predict(test)
cat_pred = catboost_reg.predict(test)
xboost_pred = xgb_reg.predict(test)
predictions = random_forest.predict(test)

In [ ]:
ensemble_preds = np.average([cat_pred, xboost_pred], weights=[0.6, 0.5], axis=0)
# Don't know what I did but it made the score worse - Khaled

In [ ]:
submission = pd.concat([patients, pd.Series(y_test, name='metastatic_diagnosis_period')], axis = 1)

#submission['metastatic_diagnosis_period'] = ensemble_preds+0.4

submission['metastatic_diagnosis_period'] = submission['metastatic_diagnosis_period'].round().astype(int)

submission.to_csv("submission.csv", index=False)

files.download('submission.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>